In [2]:
import h5py
import numpy as np
import vector
import awkward as ak
import uproot
file = h5py.File('callum_test.h5', 'r')

# Access the dataset
number_jets = np.array(file['atlas/njets'])
met = np.array(file["atlas/MET"])
eventnumber = np.array(file["atlas/eventNumber"])
jets = np.array(file["atlas/jets"])
leptons = np.array(file["atlas/leptons"])
neutrinos = np.array(file["atlas/neutrinos"])

In [3]:
lepton_pt = leptons['pt']
lepton_eta = leptons['eta']
lepton_phi = leptons['phi']
lepton_energy = leptons['energy']
lepton_charge = leptons['charge']
lepton_type = leptons['type']
jets_pt = jets['pt']
jets_eta = jets['eta']
jets_phi = jets['phi']
jets_energy = jets['energy']
jets_isbtagged = jets["is_tagged"]

In [4]:
two_b_jets_mask = np.sum(jets_isbtagged,axis=1)==2
two_opposite_leptons_mask = lepton_charge[:,0]+lepton_charge[:,1] == 0
clear_up_mask = two_opposite_leptons_mask & two_b_jets_mask
jets_which_are_btagged = jets_isbtagged[clear_up_mask]
recon_j1=vector.zip({'pt': jets_pt[clear_up_mask][jets_which_are_btagged][::2], 'eta': jets_eta[clear_up_mask][jets_which_are_btagged][::2], 'phi': jets_phi[clear_up_mask][jets_which_are_btagged][::2], 'e': jets_energy[clear_up_mask][jets_which_are_btagged][::2]})
recon_j2=vector.zip({'pt': jets_pt[clear_up_mask][jets_which_are_btagged][1::2], 'eta': jets_eta[clear_up_mask][jets_which_are_btagged][1::2], 'phi': jets_phi[clear_up_mask][jets_which_are_btagged][1::2], 'e': jets_energy[clear_up_mask][jets_which_are_btagged][1::2]})
recon_leptons=vector.zip({'pt': lepton_pt, 'eta': lepton_eta, 'phi': lepton_phi, 'e': lepton_energy})[clear_up_mask]
lepton_positive_charge_mask = lepton_charge[clear_up_mask]==1
lepton_negative_charge_mask = lepton_charge[clear_up_mask]==-1
recon_lep_positive = recon_leptons[lepton_positive_charge_mask]
recon_lep_negative = recon_leptons[lepton_negative_charge_mask]

In [5]:
#detla R
signal_delta_r_jet1_positive = recon_lep_positive.deltaR(recon_j1)
signal_delta_r_jet2_negative = recon_lep_negative.deltaR(recon_j2)
#bl invariant mass
signal_invmass_jet1_positive = (recon_j1+recon_lep_positive).m
signal_invmass_jet2_negative = (recon_j2+recon_lep_negative).m
#delta eta 
signal_delta_eta_jet1_positive = recon_lep_positive.deltaeta(recon_j1)
signal_delta_eta_jet2_negative = recon_lep_negative.deltaeta(recon_j2)
#combined pt of bl
signal_combined_pt_jet1_positive = (recon_lep_positive+recon_j1).pt
signal_combined_pt_jet2_negative = (recon_lep_negative+recon_j2).pt
#delta_r between clusters
signal_cluster_delta_r_jet1_positive = (recon_lep_positive+recon_j1).deltaR(recon_lep_negative+recon_j2)
signal_cluster_delta_r_jet2_negative = (recon_lep_positive+recon_j1).deltaR(recon_lep_negative+recon_j2)
#delta_eta between clusters
signal_cluster_delta_eta_jet1_positive = (recon_lep_positive+recon_j1).deltaeta(recon_lep_negative+recon_j2)
signal_cluster_delta_eta_jet2_negative = (recon_lep_positive+recon_j1).deltaeta(recon_lep_negative+recon_j2)
#magnitude of delta_eta + delta_R
signal_mag_de_dr_jet1_positive = np.abs(signal_delta_eta_jet1_positive + signal_delta_r_jet1_positive)
signal_mag_de_dr_jet2_negative = np.abs(signal_delta_eta_jet2_negative + signal_delta_r_jet2_negative)

#bl energy
signal_energy_jet1_positive = (recon_j1+recon_lep_positive).e
signal_energy_jet2_negative = (recon_j2+recon_lep_negative).e

#delta angle
signal_delta_angle_jet1_positive = recon_lep_positive.deltaangle(recon_j1)
signal_delta_angle_jet2_negative = recon_lep_negative.deltaangle(recon_j2)

In [6]:
h5_file = h5py.File("data.h5", 'w')
group = h5_file.create_group('Objects')
group.create_dataset("delta_r_b", data=signal_delta_r_jet1_positive[:])
group.create_dataset("bl_invmass_b", data=signal_invmass_jet1_positive[:])
group.create_dataset("delta_eta_b", data=signal_delta_eta_jet1_positive[:])
group.create_dataset("bl_pt_b", data=signal_combined_pt_jet1_positive[:])
group.create_dataset("cluster_delta_r_b", data=signal_cluster_delta_r_jet1_positive[:])
group.create_dataset("cluster_delta_eta_b", data=signal_cluster_delta_eta_jet1_positive[:])
group.create_dataset("mag_de_dr_b", data=signal_mag_de_dr_jet1_positive[:])
group.create_dataset("delta_r_bbar", data=signal_delta_r_jet2_negative[:])
group.create_dataset("bl_invmass_bbar", data=signal_invmass_jet2_negative[:])
group.create_dataset("delta_eta_bar", data=signal_delta_eta_jet2_negative[:])
group.create_dataset("bl_pt_bbar", data=signal_combined_pt_jet2_negative[:])
group.create_dataset("cluster_delta_r_bbar", data=signal_cluster_delta_r_jet2_negative[:])
group.create_dataset("cluster_delta_eta_bbar", data=signal_cluster_delta_eta_jet2_negative[:])
group.create_dataset("mag_de_dr_bbar", data=signal_mag_de_dr_jet2_negative[:])
group.create_dataset("delta_angle_b", data=signal_delta_angle_jet1_positive[:])
group.create_dataset("delta_angle_bbar", data=signal_delta_angle_jet2_negative[:])
group.create_dataset("bl_energy_b", data=signal_energy_jet1_positive[:])
group.create_dataset("bl_energy_bbar", data=signal_energy_jet2_negative[:])
h5_file.close()
print("done")


done
